In [1]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup as bs

In [ ]:
def wordlist(): # reformatting the .txt file of the book into a complete list of words
    GRE_file = open('GRE.txt').readlines()
    alphabet = list(map(chr, range(97, 123)))
    GRE_wordlist = []
    for i, word in enumerate(GRE_file):
        if word == "\n": # identifying new words using the fact that a word constitutes a line itself
            pot_new = GRE_file[i + 1].strip().replace("\n", "").replace(".", "")
            n = 0
            for characters in pot_new:
                if alphabet.count(characters) == 0:
                    n = 1
            if n == 0:
                GRE_wordlist.append(pot_new)
    return GRE_wordlist[1:len(GRE_wordlist)-1]

words = wordlist()
pd.DataFrame(words).to_csv('words.csv',index=False)
primary_sentence_matrix= []
for i,word in enumerate(words):
    website = requests.get(f"https://www.merriam-webster.com/dictionary/{word}").text
    web_parsed = bs(website,"lxml")
    try:
        sent_sect = web_parsed.find("div", id="examples")

        # this line identifies the section of the webpage that provides sample sentences

        primary_sent_ex = sent_sect.find_all("span", class_="d-block t has-aq")

        # if modifications are made to the website, change the argument given to class_ by inspecting the webpage

        primary_sent_list = []
        if primary_sent_ex:
            for sentence in primary_sent_ex:
                sentence = sentence.text
                primary_sent_list.append(sentence.replace(f'{word}', f'__{word}__'))
            primary_sentence_matrix.append(primary_sent_list)
        else: # in case the first method fails to identify any sentence
            secondary_sent_ex = sent_sect.find_all('span', class_='sub-content-thread ex-sent sents')
            for sentence in secondary_sent_ex:
                sentence = sentence.text.replace(f'{word}', f'__{word}__').replace('\n', '').replace('\t', '') + ' (sourced from Web)'
                primary_sent_list.append(sentence)
            primary_sentence_matrix.append(primary_sent_list)
        #time.sleep(3)
    except Exception:
        primary_sentence_matrix.append(["No sentence."])


In [90]:
pd.DataFrame(primary_sentence_matrix).transpose().to_csv("outputs.csv", index=False, columns=words)